<a href="https://colab.research.google.com/github/jittinabraham/Natural-Language-Processing-with-Disaster-Tweets/blob/main/Natural_Language_Processing_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!unzip '/content/nlp-getting-started.zip'

Archive:  /content/nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [17]:
import pandas as pd
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head(1000)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
995,1445,body%20bagging,NaN,Body Bagging Bitches ???? http://t.co/aFssGPnZWi,0
996,1447,body%20bagging,PURPLE BOOTH STUDIOã¢,No better feeling than seeing and being on sta...,0
997,1448,body%20bagging,Cloud 9,Mopheme and Bigstar Johnson are a problem in t...,0
998,1449,body%20bagging,NaN,I was body Bagging on the ????Today...4got to ...,0


In [18]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
import joblib
import pickle
import tensorflow
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
def clean_and_tokenize(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

In [20]:
train_df['text tokens'] = train_df['text'].apply(clean_and_tokenize)

In [26]:
train_df['text tokens'].head(100)
labels=train_df['target'].astype(int)
texts=train_df['text tokens']

In [24]:
print(labels)

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64


In [58]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

train_texts_joined = [' '.join(text) for text in texts]
# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts_joined , labels, test_size=0.2, random_state=42)

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(train_texts)

# Transform the testing data
tfidf_test = tfidf_vectorizer.transform(test_texts)

# Display the TF-IDF matrix (sparse representation)
print("TF-IDF matrix shape (train):", tfidf_train.shape)
print("TF-IDF matrix shape (test):", tfidf_test.shape)


TF-IDF matrix shape (train): (6090, 17971)
TF-IDF matrix shape (test): (1523, 17971)


In [61]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

# Create an SGD classifier
sgd_classifier = SGDClassifier(loss='log', random_state=42)

# Train the classifier on the word2vec vectors
sgd_classifier.fit(tfidf_train, train_labels)

# Predict on the testing data
predictions = sgd_classifier.predict(tfidf_test)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.8010505581089954


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


In [65]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Create an SVM classifier
svm_classifier = SVC(kernel='rbf', random_state=42)

# Train the classifier on the TF-IDF vectors
svm_classifier.fit(tfidf_train, train_labels)

# Predict on the testing data
predictions = svm_classifier.predict(tfidf_test)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.7984241628365069


In [66]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Create a Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Train the classifier on the training data
decision_tree_classifier.fit(tfidf_train, train_labels)

# Predict on the testing data
predictions = decision_tree_classifier.predict(tfidf_test)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.7156927117531189


In [33]:
!pip install gensim

In [67]:
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
word2vec_model = Word2Vec(sentences=train_texts_joined , vector_size=50, window=3, min_count=2)

# Function to transform text to word vectors
def text_to_word_vectors(text, model):
    word_vectors = []
    for word in text.split():
        if word in model.wv:
            word_vectors.append(model.wv[word])
    if not word_vectors:
        # If no word vectors found, return zero vector
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Transform training data
word2vec_train = np.array([text_to_word_vectors(text, word2vec_model) for text in train_texts])

# Transform testing data
word2vec_test = np.array([text_to_word_vectors(text, word2vec_model) for text in test_texts])

# Display the Word2Vec matrix
print("Word2Vec matrix shape (train):", word2vec_train.shape)
print("Word2Vec matrix shape (test):", word2vec_test.shape)

Word2Vec matrix shape (train): (6090, 50)
Word2Vec matrix shape (test): (1523, 50)


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the TF-IDF vectors
rf_classifier.fit(tfidf_train, train_labels)

# Predict on the testing data
predictions = rf_classifier.predict(tfidf_test)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.7839789888378201


In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the TF-IDF vectors
rf_classifier.fit(word2vec_train, train_labels)

# Predict on the testing data
predictions = rf_classifier.predict(word2vec_test)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.5712409717662508


In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Create a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()

# Train the classifier on the TF-IDF vectors
nb_classifier.fit(tfidf_train, train_labels)

# Predict on the testing data
predictions = nb_classifier.predict(tfidf_test)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.7951411687458962


In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_LEN=40
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(texts)
sequences=tokenizer_obj.texts_to_sequences(texts)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

word_index=tokenizer_obj.word_index

num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,300))

In [49]:
from tqdm import tqdm
import gensim.downloader as api
model_fasttext = api.load("glove-wiki-gigaword-300")
known_words = []
unknown_words = []
for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue

    if word in model_fasttext:
        known_words.append(word)
        embedding_matrix[i] = model_fasttext[word]
    else:
        unknown_words.append(word)

[==================================================] 100.0% 376.1/376.1MB downloaded


100%|██████████| 21271/21271 [00:00<00:00, 146867.30it/s]


In [50]:
print(len(unknown_words))
print(len(known_words))


10314
10957


In [57]:
print(unknown_words)

['mh370', '\x89û', '\x89ûò', 'prebreak', '16yr', 're\x89û', '\x89ûó', 'nowplaying', 'typhoondevastated', 'bestnaijamade', '11yearold', 'gbbo', 'disea', 'reddits', 'funtenna', 'subreddits', 'lmao', 'don\x89ûªt', 'gtgt', 'utc20150805', 'sensorsenso', '\x89ûïwhen', 'arianagrande', 'selfimage', 'sismo', 'time20150806', 'yyc', 'worldnews', 'directioners', 'spos', 'irandeal', 'shouldnt', 'trfc', 'httptcoqew4c5m1xd', 'abstorm', 'beyhive', 'rea\x89û', 'o784', 'justinbieber', 'it\x89ûªs', 'hwo', 'mediterran', 'youve', 'icemoon', 'djicemoon', 'trapmusic', 'ices\x89û', 'tcot', 'lt3', 'mhtw4fnet', 'rì©union', 'freakiest', '53inch', '300w', 'okwx', 'linkury', 'animalrescue', 'china\x89ûªs', 'you\x89ûªve', 'can\x89ûªt', 'let\x89ûªs', 'httptcoq2eblokeve', 'httptcoencmhz6y34', 'mtvhottest', 'chicagoarea', 'read\x89û', 'mikeparractor', 'wheavenly', 'tubestrike', 'standuser', 'i\x89ûªm', 'prophetmuhammad', 'by\x89û', 'meatloving', 'httptcoksawlyux02', 'be\x89û', 'igers', 'theyve', 'i77', 'viralspell', '

In [83]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
embedding_layer = tf.keras.layers.Embedding(num_words, 300, embeddings_initializer=Constant(embedding_matrix),
                                            input_length=MAX_LEN,trainable=False)
dropout = tf.keras.layers.SpatialDropout1D(0.2)
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))
lstm = tf.keras.layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)
dense = tf.keras.layers.Dense(200, activation='relu')
normalize = tf.keras.layers.BatchNormalization()

final_dropout = tf.keras.layers.Dropout(0.2)

dense_100 = tf.keras.layers.Dense(50, activation='relu')

output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

embedding_model = tf.keras.Sequential([
    embedding_layer,

    lstm,


    dense_100,
    output_layer
])
embedding_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 300)           6381600   
                                                                 
 lstm_8 (LSTM)               (None, 64)                93440     
                                                                 
 dense_16 (Dense)            (None, 50)                3250      
                                                                 
 dense_17 (Dense)            (None, 1)                 51        
                                                                 
Total params: 6478341 (24.71 MB)
Trainable params: 96741 (377.89 KB)
Non-trainable params: 6381600 (24.34 MB)
_________________________________________________________________


In [74]:
train_texts, test_texts, train_labels, test_labels = train_test_split(tweet_pad, labels, test_size=0.2, random_state=42)

In [84]:
embedding_model.compile(optimizer = 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
try:

    history = embedding_model.fit( train_texts, train_labels, batch_size=32,
                    epochs=5, verbose=1, steps_per_epoch=len(train_texts) // 32, validation_data=(test_texts, test_labels))
except RuntimeError as e:
  print(e)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


190/190 [==============================] - 19s 78ms/step - loss: 0.0000e+00 - accuracy: 0.5701 - val_loss: 0.0000e+00 - val_accuracy: 0.5739
Epoch 2/5
190/190 [==============================] - 14s 75ms/step - loss: 0.0000e+00 - accuracy: 0.5693 - val_loss: 0.0000e+00 - val_accuracy: 0.5739
Epoch 3/5
190/190 [==============================] - 14s 76ms/step - loss: 0.0000e+00 - accuracy: 0.5703 - val_loss: 0.0000e+00 - val_accuracy: 0.5739
Epoch 4/5
190/190 [==============================] - 14s 75ms/step - loss: 0.0000e+00 - accuracy: 0.5680 - val_loss: 0.0000e+00 - val_accuracy: 0.5739
Epoch 5/5
190/190 [==============================] - 14s 76ms/step - loss: 0.0000e+00 - accuracy: 0.5705 - val_loss: 0.0000e+00 - val_accuracy: 0.5739
